In [ ]:
import sys
!{sys.executable} -m pip install pafy
!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch

In [1]:
import time
from os import getenv
import os
import io
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets
import traitlets
import PIL
from PIL import ImageDraw
from IPython.display import display, Image
from jetcam.utils import bgr8_to_jpeg
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN
from IPython.display import Image
import cv2

device = torch.device('cuda')

mtcnn = MTCNN(keep_all=True, device=device)

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()
print("model loaded")

load_dotenv()
outframe = None
url = "https://youtu.be/7PYzSXHd6U4"
camera = USBCamera(width=224, height=224, capture_fps=30, capture_device=0) # confirm the capture_device number
camera.running = True
print("camera created")

#defaults
active = "Live"
default_radius_size = getenv('DEFAULT_RADIUS')
default_radius_uom = getenv('DEFAULT_RADIUS_UOM')
default_duration = getenv('DEFAULT_DURATION')
default_duration_uom = getenv('DEFAULT_DURATION_UOM')
message = None

#working values
radius_size = getenv('DEFAULT_RADIUS')
radius_uom = getenv('DEFAULT_RADIUS_UOM')
duration = getenv('DEFAULT_DURATION')
duration_uom = getenv('DEFAULT_DURATION_UOM')

def preprocess(image):
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def change_source(change):
    global active
    active = change['new']

def collate_fn(x):
    return x[0]

state_widget = ipywidgets.ToggleButtons(options=['Live', 'Playback'], value='Live')
state_widget.observe(change_source, names='value')
image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

def execute(change):
    frame = change['new']
    frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        draw = ImageDraw.Draw(frame)
        for box in boxes:
            face_only = frame.crop(box.tolist())
            processed = preprocess(face_only)
            output = model(processed)
            output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
            category_index = output.argmax()
            color = (255, 0, 0)
            if output[category_index] > 0.6:
                color = (255, 0, 0) if category_index else (0, 255, 0)
            draw.rectangle(box.tolist(), outline=color, width=6)
    buf = io.BytesIO()
    frame.save(buf, format='JPEG')
    image_widget.value = buf.getvalue()

display_handle = display(state_widget, image_widget)
while True:
    execute({'new': camera.value})
    camera.observe(execute, names='value')
    camera.unobserve_all()
    #img = cv2.rectangle(frame, start_point, end_point, color, 2)  # draw class box
    #        text = "{}:{:.2f}".format(label, score[0])
    #        cv2.putText(frame, text, start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)  # print class type with score
    #        frame = cv2.rectangle(frame, start_point, end_point, color, 2)
    #draw.rectangle(box.tolist(), outline=(255,0,0), width=6)

model loaded
camera created


ToggleButtons(options=('Live', 'Playback'), value='Live')

Image(value=b'', format='jpeg', height='224', width='224')

KeyboardInterrupt: 